Hikyuu pyecharts绘图
==============

1、引入交互式工具
-----------------
需从hikyuu.interactive引入，而不是直接从hikyuu库中引入（hikyuu是一个库，可用于编制其他的工具，而hikyuu.interactive.interactive是基于hikyuu库实现的交互式探索工具）

In [1]:
%matplotlib inline

%time from hikyuu.interactive import *
# 设置绘图引擎为echarts
use_draw_engine('echarts')

2023-10-03 22:29:18,639 [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8. [numexpr.utils::_init_num_threads]
2023-10-03 22:29:18,640 [INFO] NumExpr defaulting to 8 threads. [numexpr.utils::_init_num_threads]


std::cout are redirected to python::stdout
std::cerr are redirected to python::stderr
2023-10-03 22:29:19.362 [HKU-I] - Using SQLITE3 BaseInfoDriver (BaseInfoDriver.cpp:58)
2023-10-03 22:29:19.362 [HKU-T] - SQLITE3: D:/Hikyuu_data/stock.db (SQLiteBaseInfoDriver.cpp:32)
2023-10-03 22:29:19.363 [HKU-I] - Loading market information... (StockManager.cpp:503)
2023-10-03 22:29:19.364 [HKU-I] - Loading stock type information... (StockManager.cpp:516)
2023-10-03 22:29:19.364 [HKU-I] - Loading stock information... (StockManager.cpp:427)
2023-10-03 22:29:19.673 [HKU-I] - Loading stock weight... (StockManager.cpp:533)
2023-10-03 22:29:20.218 [HKU-I] - Loading KData... (StockManager.cpp:135)
2023-10-03 22:29:20.227 [HKU-I] - Preloading all day kdata to buffer! (StockManager.cpp:158)
2023-10-03 22:29:20.308 [HKU-I] - 0.09s Loaded Data. (StockManager.cpp:146)
CPU times: total: 1.3 s
Wall time: 3.18 s


2、创建交易系统并运行
--------------------

In [2]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(init_cash = 300000)

#创建信号指示器（相对强弱信号）
ind = RSI(C, 24)
my_sg = SG_Band(ind, 30, 70)

#固定每次买入1股
my_mm = MM_FixedCount(1)

#创建交易系统并运行
sys = SYS_Simple(tm = my_tm, sg = my_sg, mm = my_mm)
sys.set_param("buy_delay", False)
sys.set_param("sell_delay", False)
#日线信号
candle_num = 500
q = Query(-candle_num, constant.null_int64, Query.DAY)
symbol = 'sh000001'
sys.run(sm[symbol], q)

3、绘制曲线观察
---------------

In [3]:
chart = sys.plot()
k = sm[symbol].get_kdata(q)
c = CLOSE(k)
chart.render_notebook()

In [4]:
k.plot()

In [5]:
ind(c).plot(kref=k)

In [6]:
ind(c).bar(kref=k)

In [7]:
ind1 = TA_RSI(timeperiod=24)
ind2 = RSI(24)
chart = ind1(c).plot(kref=k)
ind2(c).plot(kref=k, chart=chart)
chart

4、绘制资金收益曲线
---------------------

In [8]:
#绘制资金收益曲线
x = my_tm.get_profit_curve(k.get_datetime_list(), q.ktype) # datetime is lost there
x = PRICELIST(x)
x.plot(kref=k).render_notebook()

5、回测统计报告
----------------------

In [9]:
#回测统计
from datetime import datetime

per = Performance()
print(per.report(my_tm, Datetime(datetime.today())))


帐户初始金额: 300000.00
累计投入本金: 300000.00
累计投入资产: 0.00
累计借入现金: 0.00
累计借入资产: 0.00
累计红利: 0.00
现金余额: 296372.56
未平仓头寸净值: 3276.68
当前总资产: 299649.24
已平仓交易总成本: 0.00
已平仓净利润总额: -399.85
单笔交易最大占用现金比例%: 1.18
交易平均占用现金比例%: 1.13
已平仓帐户收益率%: -0.13
帐户年复合收益率%: -0.05
帐户平均年收益率%: -0.05
赢利交易赢利总额: 0.00
亏损交易亏损总额: -399.85
已平仓交易总数: 2.00
赢利交易数: 0.00
亏损交易数: 2.00
赢利交易比例%: 0.00
赢利期望值: -199.92
赢利交易平均赢利: 0.00
亏损交易平均亏损: -199.92
平均赢利/平均亏损比例: 0.00
净赢利/亏损比例: 0.00
最大单笔赢利: 0.00
最大单笔亏损: -293.61
赢利交易平均持仓时间: 0.00
赢利交易最大持仓时间: 0.00
亏损交易平均持仓时间: 138.00
亏损交易最大持仓时间: 193.00
空仓总时间: 534.00
空仓时间/总时间%: 65.00
平均空仓时间: 267.00
最长空仓时间: 379.00
最大连续赢利笔数: 0.00
最大连续亏损笔数: 2.00
最大连续赢利金额: 0.00
最大连续亏损金额: -399.85
R乘数期望值: -0.06
交易机会频率/年: 0.90
年度期望R乘数: -0.05
赢利交易平均R乘数: 0.00
亏损交易平均R乘数: -0.06
最大单笔赢利R乘数: 0.00
最大单笔亏损R乘数: -0.09
最大连续赢利R乘数: 0.00
最大连续亏损R乘数: -0.06

